# 📊 Módulo 8 — Agregaciones y Procesado Masivo de Ficheros

En este notebook aprenderás técnicas profesionales para procesar archivos y generar estadísticas:

- Leer múltiples ficheros (`glob`, `pathlib`)
- Procesar logs grandes línea a línea
- Agregaciones estadísticas: media, percentiles, extremos
- Cargar múltiples CSV en un único DataFrame
- Generar estadísticas por grupo (por fecha, por categoría…)
- Exportar datos a CSV/JSON

---

## 1️⃣ Preparación de ficheros de ejemplo
Creamos un directorio con varios CSV simulados.

In [1]:
import os
from pathlib import Path
import pandas as pd
import numpy as np

Path('datos').mkdir(exist_ok=True)

for i in range(1,4):
    df = pd.DataFrame({
        'producto': ['A','B','C','D'],
        'ventas': np.random.randint(10,100,4),
        'precio': np.random.randint(5,20,4)
    })
    df.to_csv(f'datos/ventas_{i}.csv', index=False)

os.listdir('datos')

['ventas_3.csv', 'ventas_1.csv', 'ventas_2.csv']

---
## 2️⃣ Cargar múltiples CSV con `glob`

`glob` permite buscar ficheros por patrón:

In [2]:
import glob

rutas = glob.glob('datos/ventas_*.csv')
rutas

['datos/ventas_3.csv', 'datos/ventas_1.csv', 'datos/ventas_2.csv']

### Cargar todos los CSV en un solo DataFrame

In [3]:
df_total = pd.concat([pd.read_csv(r) for r in rutas], ignore_index=True)
df_total

,producto,ventas,precio
0,A,52,15
1,B,90,17
2,C,80,18
3,D,19,12
4,A,93,9
5,B,70,6
6,C,89,8
7,D,27,7
8,A,60,6
9,B,46,18


---
## 3️⃣ Agregaciones estadísticas

Media, mínimo, máximo, percentiles…

In [ ]:
# la salida ya es un DataFrame, podemos hacer todas las operaciones de DF que necesitems
df_total['ventas'].mean(), df_total['ventas'].max(), df_total['ventas'].quantile(0.9)

(np.float64(64.41666666666667), np.int64(93), np.float64(89.9))

Resumen completo (`describe()`):

In [5]:
df_total.describe()

,ventas,precio
count,12.000000,12.000000
mean,64.416667,11.250000
std,24.559606,4.993177
min,19.000000,5.000000
25%,50.500000,6.750000
50%,67.500000,10.500000
75%,83.750000,15.500000
max,93.000000,18.000000


---
## 4️⃣ Agrupaciones en datos cargados de múltiples ficheros

Agrupar ventas totales por producto:

In [6]:
df_total.groupby('producto')['ventas'].sum()

producto
A    205
B    206
C    251
D    111
Name: ventas, dtype: int64

Agrupaciones con varias métricas y múltiples columnas:

In [7]:
df_total.groupby('producto').agg({'ventas':['sum','mean'], 'precio':['mean','max']})

ventas                precio    
            sum       mean       mean max
producto                                 
A           205  68.333333  10.000000  15
B           206  68.666667  13.666667  18
C           251  83.666667  13.333333  18
D           111  37.000000   8.000000  12

---
## 5️⃣ Procesado de logs línea a línea

Creamos un fichero de logs de ejemplo:

In [8]:
log_text = """
INFO: user=1 evento=login tiempo=120ms
INFO: user=2 evento=login tiempo=80ms
ERROR: user=1 evento=falla tiempo=200ms
INFO: user=3 evento=login tiempo=110ms
"""

with open('datos/logs.txt', 'w') as f:
    f.write(log_text)

with open('datos/logs.txt') as f:
    for linea in f:
        print(linea.strip())


INFO: user=1 evento=login tiempo=120ms
INFO: user=2 evento=login tiempo=80ms
ERROR: user=1 evento=falla tiempo=200ms
INFO: user=3 evento=login tiempo=110ms


### Extraer información del log con regex

In [9]:
# con esto aplicamos una regex para poder ir leyendo el archivo a traves de un dataframe.
# nos permite ir leyendo p.ej. linea a linea sin afectar a la escritura de logs
import re

datos_log = []
patron = re.compile(r'user=(\d+) evento=(\w+) tiempo=(\d+)ms')

with open('datos/logs.txt') as f:
    for linea in f:
        m = patron.search(linea)
        if m:
            datos_log.append({
                'user': int(m.group(1)),
                'evento': m.group(2),
                'tiempo_ms': int(m.group(3))
            })

pd.DataFrame(datos_log)

,user,evento,tiempo_ms
0,1,login,120
1,2,login,80
2,1,falla,200
3,3,login,110


In [ ]:
#procesamiento incremental en ventana
# se va leyendo el archivo por partes, sin tener que abrir o copiar el fichero completo
df_total = pd.DataFrame()

for df_chunk ...

---
## 6️⃣ Exportar resultados

Guardar a CSV y JSON:

In [10]:
df_total.to_csv('datos/resultado.csv', index=False)
df_total.to_json('datos/resultado.json', orient='records')

os.listdir('datos')

['ventas_3.csv',
 'resultado.csv',
 'ventas_1.csv',
 'ventas_2.csv',
 'resultado.json',
 'logs.txt']

In [ ]:
#guardado segmentado por lotes (si lo que queremos guardar es muy grande)
import panda as pd

patron = re.compile(r'user=(\d+)\s+evento=(\w)\s+tiempo=(\d+)ms')

def procesar_logs_stream(ruta, chung_size=5000):
    buffer = []
    with open(ruta, encoding='utf-8') as f:
          for linea in f:
               m = patron.search(linea)
               if m:
                    buffer.append({
                         'user': int(m.group(1)),
                         'evento': m.group(2),
                         'tiempo_ms': int(m.group(3))
                    })
                if len(buffer) >= chung_size:
                    yield pd.DataFram(buffer)
                    buffer.clear()

            if buffer:
               yield pd.Dataframe(buffer)

## Guardado por lotes
for i, df_chunk in enumerate():
    df_chunk.to_csv(, index=False)
    print(f"Guardado: datos/resultado_chunk_{i:03d}.csv")

---
## 7️⃣ Ejercicio práctico

Usando los ficheros CSV generados:

### 🧩 Objetivos
1. Cargar todos los CSV del directorio en un DataFrame único
2. Calcular el total de ventas por producto
3. Calcular el precio medio por producto
4. Obtener el percentil 90 de ventas
5. Crear una tabla agregada con `agg`
6. Exportar el resultado final a JSON

Escribe tu solución abajo:

In [ ]:
# se usa 'spark' para lograr multiproceso en varios sistemas a la vez
# ojo: porque python es monohilo
dfEj = py.DataFrame()


---
## ✅ Soluciones (ocultas)

<details>
<summary>Mostrar soluciones</summary>

```python
df = pd.concat([pd.read_csv(r) for r in glob.glob('datos/ventas_*.csv')])
```

```python
df.groupby('producto')['ventas'].sum()
```

```python
df.groupby('producto')['precio'].mean()
```

```python
df['ventas'].quantile(0.9)
```

```python
tabla = df.groupby('producto').agg({'ventas':['sum','mean'], 'precio':['mean','max']})
```

```python
tabla.to_json('datos/resultado_final.json', orient='records')
```

</details>